In [82]:
# Config
# Tables
import pandas as pd
# Charting
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# Math
import numpy as np
# Tools
import os
import pickle

import plotly.io as pio
pio.renderers.default = "notebook_connected"

In [2]:
path = './data/'

# df_original = pd.read_excel(path+'Facturas.xlsx')

df_original = pd.DataFrame()
archivos = os.listdir(path)

for archivo in archivos:
    archivo_df = pd.read_excel(path + archivo)
    df_original = pd.concat([df_original, archivo_df])

df_original.head(5)

,Nº,Nº cliente,Cliente,Cód. divisa,Fecha vencimiento,Importe,Importe IVA incl.,Importe pendiente,Factor divisa,No. Serie NCF Facturas,No. Comprobante Fiscal,No. Comprobante Fiscal Rel.,RFC/Curp,Fecha registro,Fecha vencimiento NCF,Cód. almacén,Estado del documento electrónico,Nº copias impresas,Cerrado,Correctivo
0,VFR-000482,ECL0438,BANCO MULTIPLE BHD LEÓN S.A.,NaN,2022-01-31,13198.01,15573.65,0.0,0.000000,NCF FACT V,B0100000857,NaN,101136792,2022-01-31,2022-12-31,NaN,,2,True,False
1,VFR-000481,ECL0437,ASOCIACION POPULAR DE AHORROS Y PRESTAMOS,USD,2022-01-20,498.32,588.02,0.0,0.017265,NCF FACT V,B0100000856,NaN,401000131,2022-01-20,2022-12-31,NaN,,1,True,False
2,VFR-000480,ECL0615,CONSUMIDOR FINAL FISICO,USD,2022-01-20,10000.00,11800.00,8260.0,0.017265,NCF CONS F,B0200000134,NaN,00000000000,2022-01-20,NaT,NaN,,2,False,False
3,VFR-000479,ECL000770,LARC CONSULTORIA Y REDISENO ORGANIZACIONAL SRL,NaN,2022-01-19,6019.52,7103.03,0.0,0.000000,NCF FACT V,B0100000855,NaN,130109648,2022-01-19,2022-12-31,NaN,,1,True,False
4,VFR-000478,ECL0249,BANCO POPULAR DOMINICANO,USD,2022-01-14,1554.87,1834.75,0.0,0.017274,NCF FACT V,B0100000854,B0100000843,101010632,2022-01-14,2022-12-31,NaN,,1,True,False


In [3]:
df_original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 540 entries, 0 to 539
Data columns (total 20 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   Nº                                540 non-null    object        
 1   Nº cliente                        540 non-null    object        
 2   Cliente                           540 non-null    object        
 3   Cód. divisa                       408 non-null    object        
 4   Fecha vencimiento                 540 non-null    datetime64[ns]
 5   Importe                           540 non-null    float64       
 6   Importe IVA incl.                 540 non-null    float64       
 7   Importe pendiente                 540 non-null    float64       
 8   Factor divisa                     540 non-null    float64       
 9   No. Serie NCF Facturas            537 non-null    object        
 10  No. Comprobante Fiscal            503 non-null    

In [87]:
def clean(df):
    df = df_original.copy()
    df['count'] = 1

    # Cambiar Nombres
    df = df.rename(columns={
        "Fecha registro": "Fecha",
        # 'Importe': 'Valor'
        })
    df = df.sort_values(by='Fecha', ascending=False)

    # Crear Variables
    df['Mensual'] = df['Fecha'].dt.strftime('%Y-%m')
    df['Mes'] = df['Fecha'].dt.strftime('%m').astype(int)
    df['Ano'] = df['Fecha'].dt.strftime('%Y').astype(int)

    # Convertir todo a pesos
    df['Valor FOB'] = np.where(
        df['Factor divisa'] == 0, 
        df['Importe'], 
        df['Importe']/df['Factor divisa'],
    )

    return df

df = clean(df_original)
df

,Nº,Nº cliente,Cliente,Cód. divisa,Fecha vencimiento,Importe,Importe IVA incl.,Importe pendiente,Factor divisa,No. Serie NCF Facturas,...,Cód. almacén,Estado del documento electrónico,Nº copias impresas,Cerrado,Correctivo,count,Mensual,Mes,Ano,Valor FOB
0,VFR-000482,ECL0438,BANCO MULTIPLE BHD LEÓN S.A.,NaN,2022-01-31,13198.01,15573.65,0.0,0.000000,NCF FACT V,...,NaN,,2,True,False,1,2022-01,1,2022,1.319801e+04
2,VFR-000480,ECL0615,CONSUMIDOR FINAL FISICO,USD,2022-01-20,10000.00,11800.00,8260.0,0.017265,NCF CONS F,...,NaN,,2,False,False,1,2022-01,1,2022,5.792000e+05
1,VFR-000481,ECL0437,ASOCIACION POPULAR DE AHORROS Y PRESTAMOS,USD,2022-01-20,498.32,588.02,0.0,0.017265,NCF FACT V,...,NaN,,1,True,False,1,2022-01,1,2022,2.886269e+04
3,VFR-000479,ECL000770,LARC CONSULTORIA Y REDISENO ORGANIZACIONAL SRL,NaN,2022-01-19,6019.52,7103.03,0.0,0.000000,NCF FACT V,...,NaN,,1,True,False,1,2022-01,1,2022,6.019520e+03
4,VFR-000478,ECL0249,BANCO POPULAR DOMINICANO,USD,2022-01-14,1554.87,1834.75,0.0,0.017274,NCF FACT V,...,NaN,,1,True,False,1,2022-01,1,2022,9.001142e+04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
532,VFR-000012,ECL0121,CUTLER HAMMER INDUSTRIES LTD,NaN,2020-02-05,21465.00,21465.00,0.0,0.000000,NCF REG E,...,NaN,,4,True,False,1,2020-02,2,2020,2.146500e+04
530,VFR-000014,ECL0121,CUTLER HAMMER INDUSTRIES LTD,NaN,2020-02-05,105977.59,105977.59,0.0,0.000000,NCF REG E,...,NaN,,4,True,False,1,2020-02,2,2020,1.059776e+05
529,VFR-000015,ECL0416,NYPRO DR LLC,USD,2020-02-05,649.74,649.74,0.0,0.018783,NCF REG E,...,NaN,,4,True,False,1,2020-02,2,2020,3.459216e+04
528,VFR-000018,ECL0440,BANCO DE RESERVAS DE LA REPUBLICA DOMINICANA B...,USD,2020-02-05,331.06,390.65,0.0,0.018783,NCF GUB,...,NaN,,4,True,False,1,2020-02,2,2020,1.762563e+04


In [88]:
def por_Cliente(df):
    df = df.groupby(by=['Cliente']).sum()['Valor FOB'].sort_values(ascending=False)
    # px.bar(df,x=df.index,y='Valor FOB').show()
    print(len(df.index))
    print(df.index)
    px.treemap(df,path=[df.index],values='Valor FOB').show()

    return df
clientes = por_Cliente(df)

147
Index(['SCOTIA GLOBAL BUSINESS SERVICES ',
       'ASOCIACION CIBAO DE AHORROS Y PRESTAMOS',
       'ASOCIACION POPULAR DE AHORROS Y PRESTAMOS', 'CONSUMIDOR FINAL FISICO',
       'BANCO POPULAR DOMINICANO', 'EMBAJADA DE FRANCIA',
       'THE BANK OF NOVA SCOTIA SUCURSAL REPUBLICA DOMINIC',
       'PATRONATO DE CENTROS DE DIAGNOSTICOS Y MEDICINA AV',
       'MESIGAL CONSTRUCCIONES', 'BANESCO',
       ...
       'MOBA MODERN DESIGN FURNITURE', 'HENLA S A',
       'CONTRATISTAS CIVILES Y MECÁNICOS', 'BIAGGI SRL',
       'IBEROCOMERCIAL DEL CARIBE S A', 'GREEN MAN DESIGNS', 'GRUPO A M SRL',
       'NEARSHORE TELESERVICES LTD', 'L.A. CONECTOR ELECTRONICS SRL',
       ' GARCIA RODRIGUEZ Y ASOCIADOS DISENO Y CONSTRUCCIO'],
      dtype='object', name='Cliente', length=147)


In [89]:
def barchart(df, columna):
    df = df.copy()
    df = df.groupby(by=[columna, 'Mensual']).sum().reset_index()
    
    fig = px.bar(
        df,
        x = 'Mensual',
        y = 'Valor FOB',
        color = columna,
        title = f'Ventas por {columna}'
    )

    fig.show()

barchart(df, 'Cliente')

In [90]:
def barchart(df, columna):
    df = df.copy()
    df = df.groupby(by=[columna, 'Ano', 'Mes', 'Mensual']).sum().reset_index().sort_values(by=['Mensual'])
    data = []

    colors_dict = dict()
    for valor in df[columna].unique():
        r1 = np.random.randint(50,200,1)[0]
        r2 = np.random.randint(50,200,1)[0]
        r3 = np.random.randint(50,200,1)[0]
        rgb = f'rgb({r1},{r2},{r3})'
        # print(rgb)
        colors_dict[valor] = rgb
    df['color'] = df[columna].map(colors_dict)

    opacity_num = 0.4
    for i, ano in enumerate(df['Ano'].unique()):
        plot_df = df[df.Ano == ano]
        data.append(go.Bar(
            x=plot_df['Mes'], 
            y=plot_df['Valor FOB'],
            marker_color = plot_df['color'],
            hovertext = plot_df[columna],
            opacity = ((i/len(df['Ano'].unique()))*opacity_num)+opacity_num,
            name=str(ano)))

    fig = go.Figure(data=data)
    # Change the bar mode
    fig.update_layout(barmode='group')

    fig.show()

barchart(df, 'Cliente')

In [91]:
def line_total(df):
    df = df.groupby(by=['Ano','Mes']).sum().reset_index().sort_values(by=['Mes','Ano'])
    fig = px.line(
        df,
        x = 'Mes',
        y = 'Valor FOB',
        color = 'Ano',
        title = 'Ventas por Fecha',
        markers = True
        # mode = 'markers+line'
    )
    fig.show()
line_total(df)

In [92]:
def save(df):
    df.to_pickle('df.pickle')

save(df)

In [93]:
test = pd.read_pickle('df.pickle')
clies = test.Cliente.unique()
print(len(clies))
print(clies)

147
['BANCO MULTIPLE BHD LEÓN S.A.' 'CONSUMIDOR FINAL FISICO'
 'ASOCIACION POPULAR DE AHORROS Y PRESTAMOS'
 'LARC CONSULTORIA Y REDISENO ORGANIZACIONAL SRL'
 'BANCO POPULAR DOMINICANO' 'SWEDISH MATCH DOMINICANA'
 'ASOCIACION CIBAO DE AHORROS Y PRESTAMOS' 'EDWARDS LIFESCIENCES AG'
 'EMPRESA GENERADORA DE ELECTRICIDAD HAINA S A'
 'Laboratorios de Aplicaciones Médicas, SRL' 'APP QUISQUEYA SRL '
 'INFOBUS  SRL' 'LIZA ORTEGA ARQUITECTOS SRL' 'GREEN MAN DESIGNS'
 'AES ANDRES DR S A' 'GRUPO SANTOS MOREL GSM SRL'
 'PRODUCTOS ROCHE PANAMA S.A.' 'BANCO CARIBE'
 'OLIVERO RODRIGUEZ Y ASOCIADOS S R L'
 'BANCO CENTRAL DE LA REPUBLICA DOMINICANA' 'MAPFRE SALUD ARS, S.A.'
 'PASSOS COMERCIAL SRL' 'ESCUELA NACIONAL DE LA JUDICATURA'
 'MARSH FRANCO ACRA SA' 'ZONA FRANCA MULTIMODAL CAUCEDO S A'
 'COMPANIA DOMINICANA DE TELEFONOS S A' 'BANCO MULTIPLE BHD LEON SA'
 'UC UNITED CAPITAL PUESTO DE BOLSA S A'
 'SPECB SPECTRUM BRANDS DOMINICAN REPUBLIC SRL'
 'SUPERINTENDENCIA DE BANCOS' 'CORPORACION NYSSA SRL'
 '